# caQTL 2

We previously identified a proper subset of our data, let's now follow the rest of chrombpnet preprocessing

In [1]:
#chrombpnet subsets to only peaks, we can have one that is generated as peaks vs not I guess?
#do they also only subset to the peak that the QTL is in? let's see
import polars as pl
import pandas as pd
data_sus = pl.scan_csv('/data1/lesliec/sarthak/data/joint_playground/caQTL/QTD100018.all_subset.tsv', separator='\t', infer_schema_length=1000)
data_sus.fetch(10)

molecular_trait_id,chromosome,position,ref,alt,variant,ma_samples,maf,pvalue,beta,se,type,ac,an,r2,molecular_trait_object_id,gene_id,median_tpm,rsid
str,i64,i64,str,str,str,i64,f64,f64,f64,f64,str,i64,i64,str,str,str,str,str
"""Interval_1""",1,14464,"""A""","""T""","""chr1_14464_A_T…",48,0.263736,0.0712853,-0.328458,0.179592,"""SNP""",48,182,"""NA""","""Interval_1""","""Interval_1""","""NA""","""rs546169444"""
"""Interval_2""",1,14464,"""A""","""T""","""chr1_14464_A_T…",48,0.263736,0.167217,0.287697,0.206328,"""SNP""",48,182,"""NA""","""Interval_2""","""Interval_2""","""NA""","""rs546169444"""
"""Interval_3""",1,14464,"""A""","""T""","""chr1_14464_A_T…",48,0.263736,0.480256,0.108094,0.152387,"""SNP""",48,182,"""NA""","""Interval_3""","""Interval_3""","""NA""","""rs546169444"""
"""Interval_4""",1,14464,"""A""","""T""","""chr1_14464_A_T…",48,0.263736,0.805997,-0.042729,0.173386,"""SNP""",48,182,"""NA""","""Interval_4""","""Interval_4""","""NA""","""rs546169444"""
"""Interval_5""",1,14464,"""A""","""T""","""chr1_14464_A_T…",48,0.263736,0.206241,0.222002,0.174157,"""SNP""",48,182,"""NA""","""Interval_5""","""Interval_5""","""NA""","""rs546169444"""
"""Interval_6""",1,14464,"""A""","""T""","""chr1_14464_A_T…",48,0.263736,0.970258,0.005194,0.13886,"""SNP""",48,182,"""NA""","""Interval_6""","""Interval_6""","""NA""","""rs546169444"""
"""Interval_1""",1,14933,"""G""","""A""","""chr1_14933_G_A…",13,0.0714286,0.538671,0.163632,0.264955,"""SNP""",13,182,"""NA""","""Interval_1""","""Interval_1""","""NA""","""rs199856693"""
"""Interval_2""",1,14933,"""G""","""A""","""chr1_14933_G_A…",13,0.0714286,0.251985,-0.346142,0.299896,"""SNP""",13,182,"""NA""","""Interval_2""","""Interval_2""","""NA""","""rs199856693"""
"""Interval_3""",1,14933,"""G""","""A""","""chr1_14933_G_A…",13,0.0714286,0.07458,0.391881,0.2168,"""SNP""",13,182,"""NA""","""Interval_3""","""Interval_3""","""NA""","""rs199856693"""


In [2]:
#so the intervals are these set peaks I think
#and we can get the overlapping peaks with the metadata file which has the location of the peaks!!
data_sus = (
    data_sus
    .with_columns(     # create a new column lazily
        (pl.col("position") - 1).alias("position0")
    )
    .select([          # select only these columns
        "chromosome", "position0", "position", "ref", "alt", "variant",
        "ma_samples", "maf", "pvalue", "beta", "se", "type", "ac", "an", "r2",
        "molecular_trait_object_id", "gene_id", "median_tpm", "rsid",
        "molecular_trait_id"
    ])
)

In [3]:
peaks_data = pd.read_csv('/data1/lesliec/sarthak/data/joint_playground/caQTL/QTD100018_peak_metadata.tsv', sep='\t')
peaks_data

/tmp/ipykernel_72900/2038584751.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  peaks_data = pd.read_csv('/data1/lesliec/sarthak/data/joint_playground/caQTL/QTD100018_peak_metadata.tsv', sep='\t')


,phenotype_id,quant_id,group_id,gene_id,chromosome,gene_start,gene_end,strand,gene_name,gene_type,phenotype_pos,phenotype_length
0,Interval_1,Interval_1,Interval_1,Interval_1,1,10034,10617,1,Interval_1,open_chromatin,10326,584
1,Interval_2,Interval_2,Interval_2,Interval_2,1,28924,29435,1,Interval_2,open_chromatin,29180,512
2,Interval_3,Interval_3,Interval_3,Interval_3,1,180749,181528,1,Interval_3,open_chromatin,181138,780
3,Interval_4,Interval_4,Interval_4,Interval_4,1,182645,184537,1,Interval_4,open_chromatin,183591,1893
4,Interval_5,Interval_5,Interval_5,Interval_5,1,186367,187110,1,Interval_5,open_chromatin,186738,744
...,...,...,...,...,...,...,...,...,...,...,...,...
249196,Interval_249197,Interval_249197,Interval_249197,Interval_249197,Y,56861329,56862459,1,Interval_249197,open_chromatin,56861894,1131
249197,Interval_249198,Interval_249198,Interval_249198,Interval_249198,Y,56863235,56864760,1,Interval_249198,open_chromatin,56863998,1526
249198,Interval_249199,Interval_249199,Interval_249199,Interval_249199,Y,56865516,56874990,1,Interval_249199,open_chromatin,56870253,9475
249199,Interval_249200,Interval_249200,Interval_249200,Interval_249200,Y,56876239,56880531,1,Interval_249200,open_chromatin,56878385,4293


In [ ]:
peaks_data = peaks_data[["chromosome", "gene_start", "gene_end", "phenotype_pos", "phenotype_id"]]
peaks_data["chromosome"] = "chr"+peaks_data["chromosome"] .astype(str)
# peaks_data["center"]  =  ((peaks_data["gene_start"] + peaks_data["gene_end"])/2).astype(int)
# peaks_data["start"] = peaks_data["center"] - 100
# peaks_data["end"] = peaks_data["center"] + 100
peaks_data = peaks_data[['chromosome', 'gene_start', 'gene_end', "phenotype_id"]]

#literally just make it so that we subset it. Then we get chromosome and call it chrX for whatever chromosome. Then we get center by doing start and end and averaging
#then we define start and end as 100 away form the peaks... not perfect but I guess it's fine?
peaks_data

/tmp/ipykernel_72900/3427181243.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  peaks_data["chromosome"] = "chr"+peaks_data["chromosome"] .astype(str)
/tmp/ipykernel_72900/3427181243.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  peaks_data["center"]  =  ((peaks_data["gene_start"] + peaks_data["gene_end"])/2).astype(int)


,chromosome,gene_start,gene_end,phenotype_id
0,chr1,10034,10617,Interval_1
1,chr1,28924,29435,Interval_2
2,chr1,180749,181528,Interval_3
3,chr1,182645,184537,Interval_4
4,chr1,186367,187110,Interval_5
...,...,...,...,...
249196,chrY,56861329,56862459,Interval_249197
249197,chrY,56863235,56864760,Interval_249198
249198,chrY,56865516,56874990,Interval_249199
249199,chrY,56876239,56880531,Interval_249200


In [5]:
#the next step is to merge the dataframes. We will do it by getting a subset of the data
data_sus = pd.read_csv('/data1/lesliec/sarthak/data/joint_playground/caQTL/QTD100018.all_subset.tsv', sep='\t', nrows=100)
data_sus

,molecular_trait_id,chromosome,position,ref,alt,variant,ma_samples,maf,pvalue,beta,se,type,ac,an,r2,molecular_trait_object_id,gene_id,median_tpm,rsid
0,Interval_1,1,14464,A,T,chr1_14464_A_T,48,0.263736,0.071285,-0.328458,0.179592,SNP,48,182,NaN,Interval_1,Interval_1,NaN,rs546169444
1,Interval_2,1,14464,A,T,chr1_14464_A_T,48,0.263736,0.167217,0.287697,0.206328,SNP,48,182,NaN,Interval_2,Interval_2,NaN,rs546169444
2,Interval_3,1,14464,A,T,chr1_14464_A_T,48,0.263736,0.480256,0.108094,0.152387,SNP,48,182,NaN,Interval_3,Interval_3,NaN,rs546169444
3,Interval_4,1,14464,A,T,chr1_14464_A_T,48,0.263736,0.805997,-0.042729,0.173386,SNP,48,182,NaN,Interval_4,Interval_4,NaN,rs546169444
4,Interval_5,1,14464,A,T,chr1_14464_A_T,48,0.263736,0.206241,0.222002,0.174157,SNP,48,182,NaN,Interval_5,Interval_5,NaN,rs546169444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Interval_4,1,267754,T,C,chr1_267754_T_C,16,0.087912,0.499022,-0.154622,0.227640,SNP,16,182,NaN,Interval_4,Interval_4,NaN,rs367639380
96,Interval_5,1,267754,T,C,chr1_267754_T_C,16,0.087912,0.275051,-0.252686,0.229856,SNP,16,182,NaN,Interval_5,Interval_5,NaN,rs367639380
97,Interval_6,1,267754,T,C,chr1_267754_T_C,16,0.087912,0.421628,-0.147053,0.182015,SNP,16,182,NaN,Interval_6,Interval_6,NaN,rs367639380
98,Interval_7,1,267754,T,C,chr1_267754_T_C,16,0.087912,0.447159,-0.203737,0.266646,SNP,16,182,NaN,Interval_7,Interval_7,NaN,rs367639380


In [6]:
data_sus["position0"] = data_sus["position"] - 1
data_sus = data_sus[['chromosome', 'position0', 'position', 'ref', 'alt', 'variant',
       'ma_samples', 'maf', 'pvalue', 'beta', 'se', 'type', 'ac', 'an', 'r2',
       'molecular_trait_object_id', 'gene_id', 'median_tpm', 'rsid',
       'molecular_trait_id']]
data_sus

,chromosome,position0,position,ref,alt,variant,ma_samples,maf,pvalue,beta,se,type,ac,an,r2,molecular_trait_object_id,gene_id,median_tpm,rsid,molecular_trait_id
0,1,14463,14464,A,T,chr1_14464_A_T,48,0.263736,0.071285,-0.328458,0.179592,SNP,48,182,NaN,Interval_1,Interval_1,NaN,rs546169444,Interval_1
1,1,14463,14464,A,T,chr1_14464_A_T,48,0.263736,0.167217,0.287697,0.206328,SNP,48,182,NaN,Interval_2,Interval_2,NaN,rs546169444,Interval_2
2,1,14463,14464,A,T,chr1_14464_A_T,48,0.263736,0.480256,0.108094,0.152387,SNP,48,182,NaN,Interval_3,Interval_3,NaN,rs546169444,Interval_3
3,1,14463,14464,A,T,chr1_14464_A_T,48,0.263736,0.805997,-0.042729,0.173386,SNP,48,182,NaN,Interval_4,Interval_4,NaN,rs546169444,Interval_4
4,1,14463,14464,A,T,chr1_14464_A_T,48,0.263736,0.206241,0.222002,0.174157,SNP,48,182,NaN,Interval_5,Interval_5,NaN,rs546169444,Interval_5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,267753,267754,T,C,chr1_267754_T_C,16,0.087912,0.499022,-0.154622,0.227640,SNP,16,182,NaN,Interval_4,Interval_4,NaN,rs367639380,Interval_4
96,1,267753,267754,T,C,chr1_267754_T_C,16,0.087912,0.275051,-0.252686,0.229856,SNP,16,182,NaN,Interval_5,Interval_5,NaN,rs367639380,Interval_5
97,1,267753,267754,T,C,chr1_267754_T_C,16,0.087912,0.421628,-0.147053,0.182015,SNP,16,182,NaN,Interval_6,Interval_6,NaN,rs367639380,Interval_6
98,1,267753,267754,T,C,chr1_267754_T_C,16,0.087912,0.447159,-0.203737,0.266646,SNP,16,182,NaN,Interval_7,Interval_7,NaN,rs367639380,Interval_7


In [7]:
merged_dataframe = data_sus.merge(peaks_data, right_on=['phenotype_id'], left_on=["molecular_trait_id"], how='left')
merged_dataframe.head(10)
#so merges with the interval, so interval 1 when it repeats we see that same information, basically gives the chromosome and gene start and end for each one

,chromosome_x,position0,position,ref,alt,variant,ma_samples,maf,pvalue,beta,...,r2,molecular_trait_object_id,gene_id,median_tpm,rsid,molecular_trait_id,chromosome_y,gene_start,gene_end,phenotype_id
0,1,14463,14464,A,T,chr1_14464_A_T,48,0.263736,0.071285,-0.328458,...,NaN,Interval_1,Interval_1,NaN,rs546169444,Interval_1,chr1,10034,10617,Interval_1
1,1,14463,14464,A,T,chr1_14464_A_T,48,0.263736,0.167217,0.287697,...,NaN,Interval_2,Interval_2,NaN,rs546169444,Interval_2,chr1,28924,29435,Interval_2
2,1,14463,14464,A,T,chr1_14464_A_T,48,0.263736,0.480256,0.108094,...,NaN,Interval_3,Interval_3,NaN,rs546169444,Interval_3,chr1,180749,181528,Interval_3
3,1,14463,14464,A,T,chr1_14464_A_T,48,0.263736,0.805997,-0.042729,...,NaN,Interval_4,Interval_4,NaN,rs546169444,Interval_4,chr1,182645,184537,Interval_4
4,1,14463,14464,A,T,chr1_14464_A_T,48,0.263736,0.206241,0.222002,...,NaN,Interval_5,Interval_5,NaN,rs546169444,Interval_5,chr1,186367,187110,Interval_5
5,1,14463,14464,A,T,chr1_14464_A_T,48,0.263736,0.970258,0.005194,...,NaN,Interval_6,Interval_6,NaN,rs546169444,Interval_6,chr1,190802,191922,Interval_6
6,1,14932,14933,G,A,chr1_14933_G_A,13,0.071429,0.538671,0.163632,...,NaN,Interval_1,Interval_1,NaN,rs199856693,Interval_1,chr1,10034,10617,Interval_1
7,1,14932,14933,G,A,chr1_14933_G_A,13,0.071429,0.251985,-0.346142,...,NaN,Interval_2,Interval_2,NaN,rs199856693,Interval_2,chr1,28924,29435,Interval_2
8,1,14932,14933,G,A,chr1_14933_G_A,13,0.071429,0.074580,0.391881,...,NaN,Interval_3,Interval_3,NaN,rs199856693,Interval_3,chr1,180749,181528,Interval_3
9,1,14932,14933,G,A,chr1_14933_G_A,13,0.071429,0.023430,0.561551,...,NaN,Interval_4,Interval_4,NaN,rs199856693,Interval_4,chr1,182645,184537,Interval_4


In [ ]:
#this is how it would look!
#now here you filter the values to only look at the peaks that have the variant in them
merged_dataframe_n = merged_dataframe[((merged_dataframe["position"].astype(int) >= merged_dataframe["gene_start"].astype(int)) & (merged_dataframe["position"].astype(int) <= merged_dataframe["gene_end"].astype(int)))]
merged_dataframe_n #in this case just the first 100 rows don't have anything!

,chromosome_x,position0,position,ref,alt,variant,ma_samples,maf,pvalue,beta,...,r2,molecular_trait_object_id,gene_id,median_tpm,rsid,molecular_trait_id,chromosome_y,gene_start,gene_end,phenotype_id


# doing it for all

In [9]:
#idea is we'll save one that has only the peaks with the variant in them, then a separate one that has all of them
data_sus = pd.read_csv('/data1/lesliec/sarthak/data/joint_playground/caQTL/QTD100018.all_subset.tsv', sep='\t')

In [14]:
data_sus.shape

(234077507, 20)

In [ ]:
mem = data_sus.memory_usage().sum()  # in bytes
print(f"DataFrame in RAM: {mem/1024**2:.2f} MB") #yeah about right

DataFrame in RAM: 33931.52 MB


In [12]:
#now let's do the other operations
peaks_data = pd.read_csv('/data1/lesliec/sarthak/data/joint_playground/caQTL/QTD100018_peak_metadata.tsv', sep='\t')
peaks_data = peaks_data[["chromosome", "gene_start", "gene_end", "phenotype_pos", "phenotype_id"]]
peaks_data["chromosome"] = "chr"+peaks_data["chromosome"] .astype(str)
peaks_data = peaks_data[['chromosome', 'gene_start', 'gene_end', "phenotype_id"]]
peaks_data

/tmp/ipykernel_72900/3940714587.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  peaks_data = pd.read_csv('/data1/lesliec/sarthak/data/joint_playground/caQTL/QTD100018_peak_metadata.tsv', sep='\t')


,chromosome,gene_start,gene_end,phenotype_id
0,chr1,10034,10617,Interval_1
1,chr1,28924,29435,Interval_2
2,chr1,180749,181528,Interval_3
3,chr1,182645,184537,Interval_4
4,chr1,186367,187110,Interval_5
...,...,...,...,...
249196,chrY,56861329,56862459,Interval_249197
249197,chrY,56863235,56864760,Interval_249198
249198,chrY,56865516,56874990,Interval_249199
249199,chrY,56876239,56880531,Interval_249200


In [13]:
data_sus["position0"] = data_sus["position"] - 1
data_sus = data_sus[['chromosome', 'position0', 'position', 'ref', 'alt', 'variant',
       'ma_samples', 'maf', 'pvalue', 'beta', 'se', 'type', 'ac', 'an', 'r2',
       'molecular_trait_object_id', 'gene_id', 'median_tpm', 'rsid',
       'molecular_trait_id']]

In [15]:
merged_dataframe = data_sus.merge(peaks_data, right_on=['phenotype_id'], left_on=["molecular_trait_id"], how='left')

In [17]:
merged_dataframe.shape

(234077507, 24)

In [18]:
merged_dataframe.head(10)

,chromosome_x,position0,position,ref,alt,variant,ma_samples,maf,pvalue,beta,...,r2,molecular_trait_object_id,gene_id,median_tpm,rsid,molecular_trait_id,chromosome_y,gene_start,gene_end,phenotype_id
0,1,14463,14464,A,T,chr1_14464_A_T,48,0.263736,0.071285,-0.328458,...,NaN,Interval_1,Interval_1,NaN,rs546169444,Interval_1,chr1,10034,10617,Interval_1
1,1,14463,14464,A,T,chr1_14464_A_T,48,0.263736,0.167217,0.287697,...,NaN,Interval_2,Interval_2,NaN,rs546169444,Interval_2,chr1,28924,29435,Interval_2
2,1,14463,14464,A,T,chr1_14464_A_T,48,0.263736,0.480256,0.108094,...,NaN,Interval_3,Interval_3,NaN,rs546169444,Interval_3,chr1,180749,181528,Interval_3
3,1,14463,14464,A,T,chr1_14464_A_T,48,0.263736,0.805997,-0.042729,...,NaN,Interval_4,Interval_4,NaN,rs546169444,Interval_4,chr1,182645,184537,Interval_4
4,1,14463,14464,A,T,chr1_14464_A_T,48,0.263736,0.206241,0.222002,...,NaN,Interval_5,Interval_5,NaN,rs546169444,Interval_5,chr1,186367,187110,Interval_5
5,1,14463,14464,A,T,chr1_14464_A_T,48,0.263736,0.970258,0.005194,...,NaN,Interval_6,Interval_6,NaN,rs546169444,Interval_6,chr1,190802,191922,Interval_6
6,1,14932,14933,G,A,chr1_14933_G_A,13,0.071429,0.538671,0.163632,...,NaN,Interval_1,Interval_1,NaN,rs199856693,Interval_1,chr1,10034,10617,Interval_1
7,1,14932,14933,G,A,chr1_14933_G_A,13,0.071429,0.251985,-0.346142,...,NaN,Interval_2,Interval_2,NaN,rs199856693,Interval_2,chr1,28924,29435,Interval_2
8,1,14932,14933,G,A,chr1_14933_G_A,13,0.071429,0.074580,0.391881,...,NaN,Interval_3,Interval_3,NaN,rs199856693,Interval_3,chr1,180749,181528,Interval_3
9,1,14932,14933,G,A,chr1_14933_G_A,13,0.071429,0.023430,0.561551,...,NaN,Interval_4,Interval_4,NaN,rs199856693,Interval_4,chr1,182645,184537,Interval_4


In [19]:
merged_dataframe_n = merged_dataframe[((merged_dataframe["position"].astype(int) >= merged_dataframe["gene_start"].astype(int)) & (merged_dataframe["position"].astype(int) <= merged_dataframe["gene_end"].astype(int)))]

In [ ]:
merged_dataframe_n.shape #slighly smaller since we removed variants that don't agree with bim files!

(806749, 24)

In [21]:
merged_dataframe_n.columns

Index(['chromosome_x', 'position0', 'position', 'ref', 'alt', 'variant',
       'ma_samples', 'maf', 'pvalue', 'beta', 'se', 'type', 'ac', 'an', 'r2',
       'molecular_trait_object_id', 'gene_id', 'median_tpm', 'rsid',
       'molecular_trait_id', 'chromosome_y', 'gene_start', 'gene_end',
       'phenotype_id'],
      dtype='object')

In [22]:
merged_dataframe_n.to_csv('/data1/lesliec/sarthak/data/joint_playground/caQTL/QTD100018.all_subset_peaks.tsv', sep='\t', index=False)

In [23]:
#and let's save out the one with all the peaks, but first find ones whose peak is within 524288//2 base pairs
half_win = 524288 // 2
mask = (
    ((merged_dataframe['gene_end']   < merged_dataframe['position']) &
     (merged_dataframe['gene_start'] >= merged_dataframe['position'] - half_win)
    )
    |
    ((merged_dataframe['gene_start'] > merged_dataframe['position']) &
     (merged_dataframe['gene_end']   <= merged_dataframe['position'] + half_win)
    )
)

subset_peaks = merged_dataframe[mask]

In [24]:
subset_peaks.shape

(233270758, 24)

In [25]:
subset_peaks.head()

,chromosome_x,position0,position,ref,alt,variant,ma_samples,maf,pvalue,beta,...,r2,molecular_trait_object_id,gene_id,median_tpm,rsid,molecular_trait_id,chromosome_y,gene_start,gene_end,phenotype_id
0,1,14463,14464,A,T,chr1_14464_A_T,48,0.263736,0.071285,-0.328458,...,NaN,Interval_1,Interval_1,NaN,rs546169444,Interval_1,chr1,10034,10617,Interval_1
1,1,14463,14464,A,T,chr1_14464_A_T,48,0.263736,0.167217,0.287697,...,NaN,Interval_2,Interval_2,NaN,rs546169444,Interval_2,chr1,28924,29435,Interval_2
2,1,14463,14464,A,T,chr1_14464_A_T,48,0.263736,0.480256,0.108094,...,NaN,Interval_3,Interval_3,NaN,rs546169444,Interval_3,chr1,180749,181528,Interval_3
3,1,14463,14464,A,T,chr1_14464_A_T,48,0.263736,0.805997,-0.042729,...,NaN,Interval_4,Interval_4,NaN,rs546169444,Interval_4,chr1,182645,184537,Interval_4
4,1,14463,14464,A,T,chr1_14464_A_T,48,0.263736,0.206241,0.222002,...,NaN,Interval_5,Interval_5,NaN,rs546169444,Interval_5,chr1,186367,187110,Interval_5


In [30]:
#let's see if we want to remove the position or position 0 column, probably position
import numpy as np
genome_np = np.load('/data1/lesliec/sarthak/data/chrombpnet_test/hg38_tokenized.npz')

In [ ]:
genome_np['chr1'][14463:14463+2] #yeah we definitely want position 0....

array([7, 9], dtype=int8)

In [33]:
#let's check one more example
subset_peaks.iloc[13]

chromosome_x                              1
position0                             16948
position                              16949
ref                                       A
alt                                       C
variant                      chr1_16949_A_C
ma_samples                               67
maf                                0.368132
pvalue                             0.556453
beta                               0.143291
se                                 0.242579
type                                    SNP
ac                                       67
an                                      182
r2                                      NaN
molecular_trait_object_id        Interval_2
gene_id                          Interval_2
median_tpm                              NaN
rsid                            rs199745162
molecular_trait_id               Interval_2
chromosome_y                           chr1
gene_start                            28924
gene_end                        

In [34]:
genome_np['chr1'][16948]

7

In [36]:
#let's drop the position ahd chromosome_x columns
subset_peaks = subset_peaks.drop(columns=["position", "chromosome_x"])

In [37]:
subset_peaks.shape

(233270758, 22)

In [38]:
subset_peaks.to_csv('/data1/lesliec/sarthak/data/joint_playground/caQTL/QTD100018.all_subset_peaks_nearby.tsv', sep='\t', index=False)

KeyboardInterrupt: 

In [39]:
#the final thing we will do is subset it to one row per variant but then a list of peaks that are for that variant
subset_peaks.head()

,position0,ref,alt,variant,ma_samples,maf,pvalue,beta,se,type,...,r2,molecular_trait_object_id,gene_id,median_tpm,rsid,molecular_trait_id,chromosome_y,gene_start,gene_end,phenotype_id
0,14463,A,T,chr1_14464_A_T,48,0.263736,0.071285,-0.328458,0.179592,SNP,...,NaN,Interval_1,Interval_1,NaN,rs546169444,Interval_1,chr1,10034,10617,Interval_1
1,14463,A,T,chr1_14464_A_T,48,0.263736,0.167217,0.287697,0.206328,SNP,...,NaN,Interval_2,Interval_2,NaN,rs546169444,Interval_2,chr1,28924,29435,Interval_2
2,14463,A,T,chr1_14464_A_T,48,0.263736,0.480256,0.108094,0.152387,SNP,...,NaN,Interval_3,Interval_3,NaN,rs546169444,Interval_3,chr1,180749,181528,Interval_3
3,14463,A,T,chr1_14464_A_T,48,0.263736,0.805997,-0.042729,0.173386,SNP,...,NaN,Interval_4,Interval_4,NaN,rs546169444,Interval_4,chr1,182645,184537,Interval_4
4,14463,A,T,chr1_14464_A_T,48,0.263736,0.206241,0.222002,0.174157,SNP,...,NaN,Interval_5,Interval_5,NaN,rs546169444,Interval_5,chr1,186367,187110,Interval_5


In [ ]:
#let's see if the r2 column has any useful information
subset_peaks["r2"].unique()

array([nan])

In [41]:
#let's drop that too
subset_peaks = subset_peaks.drop(columns=["r2"])

: 

: 

In [ ]:
subset_peaks.head()

In [ ]:
#let's see if molecular trait object id column is identical to gene id
identical = subset_peaks["molecular_trait_object_id"].equals(subset_peaks["gene_id"])
identical

# redoing the big one

In [2]:
#we are redoing it because we have a lot of editing and processing to do!
import pandas as pd
data_sus = pd.read_csv('/data1/lesliec/sarthak/data/joint_playground/caQTL/QTD100018.all_subset.tsv', sep='\t')
peaks_data = pd.read_csv('/data1/lesliec/sarthak/data/joint_playground/caQTL/QTD100018_peak_metadata.tsv', sep='\t')
peaks_data = peaks_data[["chromosome", "gene_start", "gene_end", "phenotype_pos", "phenotype_id"]]
peaks_data["chromosome"] = "chr"+peaks_data["chromosome"] .astype(str)
peaks_data = peaks_data[['chromosome', 'gene_start', 'gene_end', "phenotype_id"]]

/tmp/ipykernel_103818/1299764431.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  peaks_data = pd.read_csv('/data1/lesliec/sarthak/data/joint_playground/caQTL/QTD100018_peak_metadata.tsv', sep='\t')


In [3]:
data_sus["position0"] = data_sus["position"] - 1
data_sus = data_sus[['chromosome', 'position0', 'position', 'ref', 'alt', 'variant',
       'ma_samples', 'maf', 'pvalue', 'beta', 'se', 'type', 'ac', 'an', 'r2',
       'molecular_trait_object_id', 'gene_id', 'median_tpm', 'rsid',
       'molecular_trait_id']]

In [4]:
merged_dataframe = data_sus.merge(peaks_data, right_on=['phenotype_id'], left_on=["molecular_trait_id"], how='left')

In [5]:
half_win = 524288 // 2
mask = (
    ((merged_dataframe['gene_end']   < merged_dataframe['position']) &
     (merged_dataframe['gene_start'] >= merged_dataframe['position'] - half_win)
    )
    |
    ((merged_dataframe['gene_start'] > merged_dataframe['position']) &
     (merged_dataframe['gene_end']   <= merged_dataframe['position'] + half_win)
    )
)

subset_peaks = merged_dataframe[mask]

In [6]:
#let's see if we need to drop anymore peaks first
#the type column should just be snp
subset_peaks["type"].unique()

array(['SNP'], dtype=object)

In [7]:
#the median tpm should be nan as well
subset_peaks["median_tpm"].unique()

array([nan])

In [ ]:
#these column might all be identical, molecular_trait_object_id, gene_id, phenotype_id and molecular_trait_id
identical = subset_peaks["gene_id"].equals(subset_peaks["molecular_trait_object_id"])
print(identical)
identical = subset_peaks["gene_id"].equals(subset_peaks["phenotype_id"])
print(identical)
identical = subset_peaks["gene_id"].equals(subset_peaks["molecular_trait_id"])
print(identical)
#they are all identical, so we can drop all of them

True
True
True


3

In [11]:
subset_peaks = subset_peaks.drop(columns=["position", "chromosome_x", "r2", "type", "median_tpm", "phenotype_id", "molecular_trait_object_id", "molecular_trait_id"])
subset_peaks.shape

(233270758, 16)

In [12]:
subset_peaks.head()

,position0,ref,alt,variant,ma_samples,maf,pvalue,beta,se,ac,an,gene_id,rsid,chromosome_y,gene_start,gene_end
0,14463,A,T,chr1_14464_A_T,48,0.263736,0.071285,-0.328458,0.179592,48,182,Interval_1,rs546169444,chr1,10034,10617
1,14463,A,T,chr1_14464_A_T,48,0.263736,0.167217,0.287697,0.206328,48,182,Interval_2,rs546169444,chr1,28924,29435
2,14463,A,T,chr1_14464_A_T,48,0.263736,0.480256,0.108094,0.152387,48,182,Interval_3,rs546169444,chr1,180749,181528
3,14463,A,T,chr1_14464_A_T,48,0.263736,0.805997,-0.042729,0.173386,48,182,Interval_4,rs546169444,chr1,182645,184537
4,14463,A,T,chr1_14464_A_T,48,0.263736,0.206241,0.222002,0.174157,48,182,Interval_5,rs546169444,chr1,186367,187110


In [13]:
peaks_data

,chromosome,gene_start,gene_end,phenotype_id
0,chr1,10034,10617,Interval_1
1,chr1,28924,29435,Interval_2
2,chr1,180749,181528,Interval_3
3,chr1,182645,184537,Interval_4
4,chr1,186367,187110,Interval_5
...,...,...,...,...
249196,chrY,56861329,56862459,Interval_249197
249197,chrY,56863235,56864760,Interval_249198
249198,chrY,56865516,56874990,Interval_249199
249199,chrY,56876239,56880531,Interval_249200


In [14]:
#the other thing we'll do is remove ma_samples, gene start and gene end columns
subset_peaks = subset_peaks.drop(columns=["ma_samples", "gene_start", "gene_end"])

In [15]:
#now let's group by and then make lists
#can find number of elements in the group by checking length of the list!

agg_dict = {
    'position0':     'first',
    'ref':           'first',
    'alt':           'first',
    'maf':           'first',
    'ac':            'first',
    'an':            'first',
    'chromosome_y':  'first',
    'rsid':          'first',
    'pvalue':        list,
    'beta':          list,
    'gene_id':       list,
}

grouped = subset_peaks \
    .groupby('variant', as_index=False) \
    .agg(agg_dict)
grouped.shape

(6508629, 12)

In [16]:
grouped.head()

,variant,position0,ref,alt,maf,ac,an,chromosome_y,rsid,pvalue,beta,gene_id
0,chr10_100000235_C_T,100000234,C,T,0.340659,62,182,chr10,rs11596870,"[0.719412, 0.113877, 0.953659, 0.00137416, 3.8...","[0.0512266, -0.237318, 0.00746471, -0.353161, ...","[Interval_26170, Interval_26171, Interval_2617..."
1,chr10_100000943_G_A,100000942,G,A,0.076923,14,182,chr10,rs11190359,"[0.0224591, 0.0439213, 0.129788, 0.584645, 0.5...","[-0.622646, 0.585245, -0.375654, -0.12116, -0....","[Interval_26170, Interval_26171, Interval_2617..."
2,chr10_100000979_T_C,100000978,T,C,0.032967,6,182,chr10,rs11190360,"[0.0172942, 0.735536, 0.029347, 0.0188991, 0.4...","[-0.95319, 0.146098, -0.787944, -0.752098, 0.3...","[Interval_26170, Interval_26171, Interval_2617..."
3,chr10_100002012_T_C,100002011,T,C,0.043956,8,182,chr10,rs11190362,"[0.412635, 0.0255791, 0.955902, 0.199178, 0.15...","[-0.283286, 0.807535, -0.0172548, 0.353885, -0...","[Interval_26170, Interval_26171, Interval_2617..."
4,chr10_100002628_A_C,100002627,A,C,0.461538,84,182,chr10,rs11190363,"[0.143933, 0.566345, 0.329146, 0.000515478, 0....","[-0.198788, -0.0832435, -0.119994, -0.36588, 0...","[Interval_26170, Interval_26171, Interval_2617..."


In [17]:
#now let's save this out!
grouped.to_csv('/data1/lesliec/sarthak/data/joint_playground/caQTL/QTD100018.all_subset_peaks_nearby.tsv', sep='\t', index=False)

In [18]:
#let's load it in and make sure it's good
grouped2 = pd.read_csv('/data1/lesliec/sarthak/data/joint_playground/caQTL/QTD100018.all_subset_peaks_nearby.tsv', sep='\t', nrows=100)
grouped2.head()

,variant,position0,ref,alt,maf,ac,an,chromosome_y,rsid,pvalue,beta,gene_id
0,chr10_100000235_C_T,100000234,C,T,0.340659,62,182,chr10,rs11596870,"[0.719412, 0.113877, 0.953659, 0.00137416, 3.8...","[0.0512266, -0.237318, 0.00746471, -0.353161, ...","['Interval_26170', 'Interval_26171', 'Interval..."
1,chr10_100000943_G_A,100000942,G,A,0.076923,14,182,chr10,rs11190359,"[0.0224591, 0.0439213, 0.129788, 0.584645, 0.5...","[-0.622646, 0.585245, -0.375654, -0.12116, -0....","['Interval_26170', 'Interval_26171', 'Interval..."
2,chr10_100000979_T_C,100000978,T,C,0.032967,6,182,chr10,rs11190360,"[0.0172942, 0.735536, 0.029347, 0.0188991, 0.4...","[-0.95319, 0.146098, -0.787944, -0.752098, 0.3...","['Interval_26170', 'Interval_26171', 'Interval..."
3,chr10_100002012_T_C,100002011,T,C,0.043956,8,182,chr10,rs11190362,"[0.412635, 0.0255791, 0.955902, 0.199178, 0.15...","[-0.283286, 0.807535, -0.0172548, 0.353885, -0...","['Interval_26170', 'Interval_26171', 'Interval..."
4,chr10_100002628_A_C,100002627,A,C,0.461538,84,182,chr10,rs11190363,"[0.143933, 0.566345, 0.329146, 0.000515478, 0....","[-0.198788, -0.0832435, -0.119994, -0.36588, 0...","['Interval_26170', 'Interval_26171', 'Interval..."


In [ ]:
#yeah it looks fine!